# Figure 8

In [2]:
%run 'flow_fig_init.py'

In [7]:
n_features = 6

titles = {
    "Britned": [
        "Integral (GB)<br>",
        "Britned scheduled inflow<br>(GB) [GW]",
        "Other generation<br>(CE) [GW]",
        "Waste generation<br>(CE) [GW]",
        "Gas generation<br>(GB) [GW]",
        "Gas generation<br>(CE) [GW]",
    ],
    "Kontiskan": [
        "Kontiskan scheduled inflow<br>(Nordic) [GW]",
        "Forecast error generation<br>(Nordic) [GW]",
        "Offshore Wind generation<br>(CE) [GW]",
        "Lignite generation<br>(CE) [GW]",
        "Integral (Nordic)<br>",
        "Solar generation<br>(Nordic) [GW]",
    ],
}

factors = {
    "Britned": [1, 0.001, 0.001, 0.001, 0.001, 0.001],
    "Kontiskan": [0.001, 0.001, 0.001, 0.001, 1, 0.001],
}

colors = {
    "Britned": [
        px.colors.qualitative.T10[2],
        px.colors.qualitative.T10[4],
        px.colors.qualitative.T10[0],
        px.colors.qualitative.T10[0],
        px.colors.qualitative.T10[0],
        px.colors.qualitative.T10[0],
    ],
    "Kontiskan": [
        px.colors.qualitative.T10[4],
        px.colors.qualitative.T10[0],
        px.colors.qualitative.T10[0],
        px.colors.qualitative.T10[0],
        px.colors.qualitative.T10[2],
        px.colors.qualitative.T10[0],
    ],
}
# colors = {
#     'Britned': ['rgb(255,0,81)', 'rgb(64,141,55)', 'rgb(0,139,251)', 'rgb(0,139,251)', 'rgb(0,139,251)', 'rgb(0,139,251)'],
#     'Kontiskan': ['rgb(64,141,55)', 'rgb(0,139,251)', 'rgb(0,139,251)', 'rgb(0,139,251)', 'rgb(0,139,251)', 'rgb(255,0,81)']
#     }

mt = "_full"
# fig = make_subplots(rows=len(targets[area]), cols=n_features, row_titles=targets[area])
fig = make_subplots(
    rows=n_features // 2,
    cols=4,
    horizontal_spacing=0.05,
    vertical_spacing=0.14,
    specs=[
        [{"l": 0.02}, {"r": 0.02}, {"l": 0.02}, {"r": 0.02}],
        [{"l": 0.02}, {"r": 0.02}, {"l": 0.02}, {"r": 0.02}],
        [{"l": 0.02}, {"r": 0.02}, {"l": 0.02}, {"r": 0.02}],
    ],
)

fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode="markers",
        marker_size=40,
        marker_color=px.colors.qualitative.T10[2],
        name="Frequency Indicator  ",
        legendgroup=1,
        legendgrouptitle_text="<b>Feature type</b>",
    ),
    col=1,
    row=1,
)
fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode="markers",
        marker_size=40,
        marker_color=px.colors.qualitative.T10[4],
        name="HVDC Flow  ",
        legendgroup=2,
        legendgrouptitle_text=" ",
    ),
    col=1,
    row=1,
)
fig.add_trace(
    go.Scatter(
        x=[None],
        y=[None],
        mode="markers",
        marker_size=40,
        marker_color=px.colors.qualitative.T10[0],
        name="Generation/Load  ",
        legendgroup=3,
        legendgrouptitle_text=" ",
    ),
    col=1,
    row=1,
)


# fig.update_layout(font=dict(size=35, color='black'), margin_t=180, margin_b=350)
# fig.update_annotations(font_size=35)
# fig.update_layout(legend_font_size=35)
fig.update_layout(font=dict(size=17, color="black"))
fig.update_annotations(font_size=17)
fig.update_layout(legend_font_size=20)

# , vertical_spacing = 0.2)
for i, [area, targ] in enumerate([["GB-CE", "Britned"], ["Nordic-CE", "Kontiskan"]]):
    X_test = pd.read_hdf(
        version_folder.format(area, targ, version) + "X_test{}_links.h5".format(mt)
    )
    shap_vals = shap_vals_df(area, version, targ, model_type=mt)
    important_features = most_important_shap(
        area, version, targ, n_features=n_features, model_type=mt
    )
    print(important_features)
    for j, feature in enumerate(important_features):
        fig.add_trace(
            go.Scatter(
                x=X_test[feature] * factors[targ][j],
                y=shap_vals[feature],
                text=X_test.index,
                mode="markers",
                marker_size=4,
                marker_opacity=0.75,
                marker_color=colors[targ][j],
                # marker_opacity=0.95,
                # marker_color=X_test.index.astype(int)/10**12,
                # marker_colorscale='RdYlBu',
                # marker_showscale=True,
                showlegend=False,
            ),  # , marker_colorbar={'title': input_col_names[color_feature]}))
            col=(i * 2) + (j % 2 + 1),
            row=(j // 2) + 1,
        )

        if (i * 2) + (j % 2 + 1) == 1:
            fig.update_layout(
                {
                    "yaxis{}".format(j + (j // 2) * 2 + 1 + (i * 2)): {
                        "title_standoff": 12,
                        "title": "SHAP value<br>(unscheduled Outflow) [MW]".format(
                            targ
                        ),
                        "linewidth": 1,
                        "linecolor": "black",
                        "mirror": True,
                        "ticks": "outside",
                        "showline": True,
                    },
                    "xaxis{}".format(j + (j // 2) * 2 + 1 + (i * 2)): {
                        "title_standoff": 12,
                        "title": {"text": titles[targ][j]},
                        "linewidth": 1,
                        "linecolor": "black",
                        "mirror": True,
                        "ticks": "outside",
                        "showline": True,
                    },
                }
            )
        else:
            fig.update_layout(
                {
                    "yaxis{}".format(j + (j // 2) * 2 + 1 + (i * 2)): {
                        "title_standoff": 12,
                        "linewidth": 1,
                        "linecolor": "black",
                        "mirror": True,
                        "ticks": "outside",
                        "showline": True,
                    },
                    "xaxis{}".format(j + (j // 2) * 2 + 1 + (i * 2)): {
                        "title_standoff": 12,
                        "title": {"text": titles[targ][j]},
                        "linewidth": 1,
                        "linecolor": "black",
                        "mirror": True,
                        "ticks": "outside",
                        "showline": True,
                    },
                }
            )
        if j + (j // 2) * 2 + 1 + (i * 2) == 1:
            fig.add_annotation(
                x=0.05,
                y=1,
                text="<b>{}</b>".format(j + 1),
                showarrow=False,
                xanchor="left",
                xref="x domain".format(j + (j // 2) * 2 + 1 + (i * 2)),
                yref="y domain".format(j + (j // 2) * 2 + 1 + (i * 2)),
                font_size=30,
            )
        else:
            fig.add_annotation(
                x=0.05,
                y=1,
                text="<b>{}</b>".format(j + 1),
                showarrow=False,
                xanchor="left",
                xref="x{} domain".format(j + (j // 2) * 2 + 1 + (i * 2)),
                yref="y{} domain".format(j + (j // 2) * 2 + 1 + (i * 2)),
                font_size=30,
            )


# fig.update_layout(margin_t=0)
fig.update_layout(margin_r=50)
fig.update_layout(
    template="plotly_white", width=1240, height=1250, legend_orientation="h"
)


# fig.add_annotation(x=0.45, y=0,
#             xref='paper',
#             yref='paper',
#             text="Text annotation without arrow",
#             showarrow=True,
#             arrowhead=0,
#             yshift=1000,
#             arrowcolor="black",
#             arrowsize=3,
#             arrowwidth=1,)


fig.update_layout(
    legend=dict(
        title=" ",
        # title='<b>Feature type</b>',
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5,
        borderwidth=1,
        bordercolor="black",
        # xref='paper',
        # yref='paper'
    )
)


fig.add_annotation(
    x=0.23,
    y=1.08,
    text="<b>Britned</b>",
    showarrow=False,
    xanchor="center",
    xref="paper",
    yref="paper",
    font_size=40,
)
fig.add_annotation(
    x=0.77,
    y=1.08,
    text="<b>Kontiskan</b>",
    showarrow=False,
    xanchor="center",
    xref="paper",
    yref="paper",
    font_size=40,
)

fig.add_annotation(
    x=0.49,
    y=-0.15,
    # text='<b>Kontiskan</b>',
    showarrow=True,
    xanchor="center",
    yanchor="bottom",
    xref="paper",
    yref="paper",
    font_size=60,
    arrowwidth=1,
    ax=0,
    ay=-6000,
    arrowcolor="black",
)


print("creating plot".format(area))
scope = pio.kaleido.scope
scope._shutdown_kaleido()
pio.write_image(
    fig, "fig8.pdf".format(targ, mt), format="pdf", validate=False, engine="kaleido"
)
time.sleep(1)
pio.write_image(
    fig, "fig8.pdf".format(targ, mt), format="pdf", validate=False, engine="kaleido"
)
scope._shutdown_kaleido()


['f_integral_GB', 'Britned', 'gen_other_CE', 'gen_waste', 'gen_gas_GB', 'gen_gas_CE']
['Kontiskan', 'forecast_error_total_gen_Nordic', 'gen_wind_off_CE', 'gen_lignite', 'f_integral_Nordic', 'gen_solar_Nordic']
creating plot
